# Lightweight Fine-Tuning Project

* PEFT technique: LoRA
* Model: GPT-2 and DistilBert
* Evaluation approach: F1-Score
* Fine-tuning dataset: GLUE QQP

In [1]:
# import os
# import zipfile

# def zip_directory(folder_path, output_path):
#     """
#     Compresses the specified folder into a ZIP file.

#     :param folder_path: Path to the folder to compress.
#     :param output_path: Path where the ZIP file will be saved.
#     """
#     with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, dirs, files in os.walk(folder_path):
#             for file in files:
#                 file_path = os.path.join(root, file)
#                 # Add file to zip, preserving the folder structure
#                 zipf.write(file_path, os.path.relpath(file_path, folder_path))
#     print(f"Folder '{folder_path}' has been zipped successfully as '{output_path}'.")

# # Specify the directory you want to zip
# folder_to_zip = '.'  # Current directory

# # Specify the output ZIP file name
# output_zip = 'my_project.zip'

# # Call the function to create the ZIP archive
# zip_directory(folder_to_zip, output_zip)

Folder '.' has been zipped successfully as 'my_project.zip'.


In [2]:
# from IPython.display import FileLink, display

# # Create a FileLink object
# link = FileLink(r'my_project.zip')

# # Display the link
# display(link)


/workspace/my_project.zip

In [4]:
!pip install transformers datasets peft accelerate bitsandbytes scikit-learn
!pip install --upgrade datasets huggingface_hub transformers peft
!pip install transformers[torch]

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 71.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 301.8/301.8 kB 30.7 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 90.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 42.6 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.21.4
    Uninstalling huggingface-hub-0.21.4:
      Successfully uninstalled huggingface-hub-0.21.4
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this dir

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


## Libraries and Load Data set

In [1]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer
)
from peft import LoraConfig, TaskType, get_peft_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 1. Load the QQP dataset from GLUE
dataset = load_dataset("glue", "qqp")
train_dataset_raw = dataset["train"]
eval_dataset_raw = dataset["validation"]

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

README.md:   0%|          | 0.00/35.3k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/33.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/3.73M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/36.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/363846 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/40430 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/390965 [00:00<?, ? examples/s]

In [2]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
    # Convert logits -> NumPy if they’re not already
    if isinstance(logits, torch.Tensor):
        logits = logits.detach().cpu().numpy()
    
    # Convert labels -> NumPy if they’re not already
    if isinstance(labels, torch.Tensor):
        labels = labels.detach().cpu().numpy()

    # Now do argmax in NumPy
    preds = np.argmax(logits, axis=-1)

    # Evaluate
    acc  = accuracy_score(labels, preds)
    prec = precision_score(labels, preds, average="weighted", zero_division=0)
    rec  = recall_score(labels, preds, average="weighted", zero_division=0)
    f1   = f1_score(labels, preds, average="weighted", zero_division=0)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [ ]:
model_name = "gpt2" 
# We'll also need a tokenizer:
tokenizer = AutoTokenizer.from_pretrained(model_name)
# GPT-2 doesn't have a pad token, so we reuse the EOS token
tokenizer.pad_token = tokenizer.eos_token

MAX_LENGTH = 128

def preprocess_function(examples):
    q1_list = examples["question1"]
    q2_list = examples["question2"]
    texts = [f"{q1} [SEP] {q2}" for q1, q2 in zip(q1_list, q2_list)]
    
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )
    
    # QQP labels are 0 or 1
    tokenized["labels"] = examples["label"]
    return tokenized

train_dataset = train_dataset_raw.map(preprocess_function, batched=True)
eval_dataset  = evawl_dataset_raw.map(preprocess_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [4]:
# Specify the number of labels (2 for QQP's binary classification)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# For safety, ensure token embeddings match the tokenizer size
model.resize_token_embeddings(len(tokenizer))

model.config.pad_token_id = tokenizer.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [55]:
training_args = TrainingArguments(
    output_dir="./gpt2-qqp",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    num_train_epochs=1,          # Increase for better results
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # If you have a GPU that supports mixed-precision
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

/home/student/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [57]:
metrics = trainer.evaluate()
pd.DataFrame(pd.Series(metrics))

,0
eval_loss,4.233539
eval_model_preparation_time,0.004200
eval_accuracy,0.631511
eval_precision,0.500734
eval_recall,0.631511
eval_f1,0.489489
eval_runtime,412.688900
eval_samples_per_second,97.967000
eval_steps_per_second,48.984000


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [58]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    target_modules=[
        "attn.c_attn",
        "attn.c_proj",
        "mlp.c_fc",
        "mlp.c_proj"
    ]
)

# Wrap the original model with LoRA
lora_model = get_peft_model(model, lora_config)
trainable_params = sum(p.numel() for p in lora_model.parameters() if p.requires_grad)
print(f"Number of trainable parameters: {trainable_params}")

Number of trainable parameters: 1181184


/home/student/.local/lib/python3.10/site-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [66]:
training_args = TrainingArguments(
    output_dir="./gpt2-lora-qqp",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    num_train_epochs=1,          # Increase for better results
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),  # If you have a GPU that supports mixed-precision
    report_to="none"
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset.select(range(10000)),
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()

/home/student/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.891100,0.800575,0.772817,0.784349,0.772817,0.775737


TrainOutput(global_step=5000, training_loss=0.8983977416992187, metrics={'train_runtime': 1232.3104, 'train_samples_per_second': 8.115, 'train_steps_per_second': 4.057, 'total_flos': 662313369600000.0, 'train_loss': 0.8983977416992187, 'epoch': 1.0})

Save PEFT model

In [15]:
lora_model.save_pretrained("gpt-lora-qqp", safe_serialization=True)

Load PEFT model

In [7]:
from peft import PeftModel

base_model = AutoModelForSequenceClassification.from_pretrained("gpt2", num_labels=2)
new_lora_model = PeftModel.from_pretrained(
    base_model,
    "gpt-lora-qqp",
    torch_dtype=torch.float32,  # or "auto"
    device_map={"": "cpu"},     # load everything onto CPU first
)
new_lora_model = new_lora_model.to("cuda")  # then move to GPU

# 3. Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # reuse eos as pad

# 4. IMPORTANT: Set the model config pad_token_id
new_lora_model.config.pad_token_id = tokenizer.eos_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
training_args = TrainingArguments(
    output_dir="./lora-gpt2-qqp-eval",
    per_device_eval_batch_size=8,
    do_train=False,      # We're not training here
    do_eval=True,        # We want to evaluate
    logging_strategy="no"
)

trainer = Trainer(
    model=new_lora_model,
    args=training_args,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

In [9]:
metrics = trainer.evaluate()
pd.DataFrame(pd.Series(metrics))

,0
eval_loss,0.800711
eval_model_preparation_time,0.011700
eval_accuracy,0.772792
eval_precision,0.784318
eval_recall,0.772792
eval_f1,0.775712
eval_runtime,362.641400
eval_samples_per_second,111.488000
eval_steps_per_second,13.937000


# Repeat Exercise with DistilBERT

In [29]:
model_name = "distilbert-base-uncased"

In [30]:
# Load DistilBERT tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
distilbert = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# We'll also need a tokenizer:
tokenizer = AutoTokenizer.from_pretrained(model_name)

MAX_LENGTH = 128

def preprocess_function(examples):
    q1_list = examples["question1"]
    q2_list = examples["question2"]
    texts = [f"{q1} [SEP] {q2}" for q1, q2 in zip(q1_list, q2_list)]
    
    tokenized = tokenizer(
        texts,
        truncation=True,
        padding="max_length",
        max_length=MAX_LENGTH
    )
    
    # QQP labels are 0 or 1
    tokenized["labels"] = examples["label"]
    return tokenized

train_dataset = train_dataset_raw.map(preprocess_function, batched=True)
eval_dataset  = eval_dataset_raw.map(preprocess_function, batched=True)

train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
eval_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map:   0%|          | 0/363846 [00:00<?, ? examples/s]

Map:   0%|          | 0/40430 [00:00<?, ? examples/s]

In [36]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./distilbert-qqp",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    num_train_epochs=1,          # Increase for better results
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=distilbert,
    args=training_args,
    train_dataset=train_dataset.select(range(10000)),
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

#no training
# metrics = trainer.evaluate()
# pd.DataFrame(pd.Series(metrics))

/home/student/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [37]:
# Fine-tune DistilBERT (full-fine tuning)
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.434200,0.482145,0.780707,0.793140,0.780707,0.783650


TrainOutput(global_step=2500, training_loss=0.4985191879272461, metrics={'train_runtime': 335.1983, 'train_samples_per_second': 29.833, 'train_steps_per_second': 7.458, 'total_flos': 337980979200000.0, 'train_loss': 0.4985191879272461, 'epoch': 1.0})

In [31]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS,
    # DistilBERT has "q_lin", "k_lin", "v_lin", "out_lin" in each multi-head attention,
    # and "fc" in the feed-forward. We'll specify these in target_modules.
    # If you omit target_modules, PEFT will attempt to apply LoRA to all linear layers it finds.
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin", "fc"]
)

# Wrap the DistilBERT model with LoRA
lora_model = get_peft_model(distilbert, lora_config)
#print("Trainable parameters:", lora_model.num_trainable_parameters())

In [32]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./distilbert-lora-qqp",
    evaluation_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="epoch",
    num_train_epochs=1,          # Increase for better results
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    fp16=torch.cuda.is_available(),
    report_to="none"
)

trainer = Trainer(
    model=lora_model,
    args=training_args,
    train_dataset=train_dataset.select(range(10000)),
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

# Fine-tune DistilBERT + LoRA
trainer.train()

/home/student/.local/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.440800,0.487365,0.763814,0.779532,0.763814,0.767302


TrainOutput(global_step=2500, training_loss=0.5558701385498047, metrics={'train_runtime': 334.4992, 'train_samples_per_second': 29.895, 'train_steps_per_second': 7.474, 'total_flos': 337980979200000.0, 'train_loss': 0.5558701385498047, 'epoch': 1.0})

Save PEFT model

In [33]:
lora_model.save_pretrained("distilbert-lora-qqp")

Load PEFT model

In [34]:
from transformers import AutoModelForSequenceClassification
from peft import PeftModel

model_name = "distilbert-base-uncased"
# Load the base model
base_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
# Load the LoRA adapters into the base model
new_lora_model = PeftModel.from_pretrained(base_model, "distilbert-lora-qqp")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:


training_args = TrainingArguments(
    output_dir="./lora-distilbert-qqp-eval",
    per_device_eval_batch_size=8,
    do_train=False,      # We're not training here
    do_eval=True,        # We want to evaluate
    logging_strategy="no"
)

trainer = Trainer(
    model=new_lora_model,
    args=training_args,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

metrics = trainer.evaluate()
pd.DataFrame(pd.Series(metrics))

,0
eval_loss,0.487365
eval_model_preparation_time,0.006400
eval_accuracy,0.763814
eval_precision,0.779532
eval_recall,0.763814
eval_f1,0.767302
eval_runtime,95.092300
eval_samples_per_second,425.166000
eval_steps_per_second,53.148000
